In [2]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import re
import numpy as np

In [3]:
#importing the excel spreadsheet containing my police data as a csv. Also using .head() and .info() to take a look at all the rows in the dataset to prepare for cleaning. 

df = pd.read_csv(r"C:\Users\kesha\Documents\nss_da10\projects\police-response-capstone\data\Metro_Nashville_Police_Department_Incidents_20240217.csv")

In [4]:
df.head()

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Weapon Description,Victim Number,Domestic Related,Victim Type,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Mapped Location
0,20190496213_11,20190496213,D,DISPATCHED,U,UNFOUNDED,Closed,7/1/2019 17:31,7/1/2019 18:17,2410 2410,...,NONE,1,True,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.696 36.218)
1,20180290646_11,20180290646,D,DISPATCHED,A,CLEARED BY ARREST,Closed,3/31/2018 12:21,3/31/2018 19:41,5824 5824,...,NONE,1,False,B,BUSINESS,NaN,NaN,NaN,NaN,POINT (-86.713 36.042)
2,20200243837_11,20200243837,D,DISPATCHED,U,UNFOUNDED,Closed,4/6/2020 11:15,4/6/2020 13:11,3144 3144,...,NONE,1,False,G,GOVERNMENT,NaN,NaN,NaN,NaN,POINT (-86.759 36.232)
3,20170471747_11,20170471747,D,DISPATCHED,U,UNFOUNDED,Closed,5/29/2017 2:33,5/29/2017 4:27,1714 1714,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.907 36.086)
4,20171099391_21,20171099391,D,DISPATCHED,A,CLEARED BY ARREST,Closed,12/16/2017 18:30,12/16/2017 22:44,1101 1101,...,PERSONAL (HANDS),1,True,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,POINT (-86.586 36.178)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865175 entries, 0 to 865174
Data columns (total 31 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Primary Key                  865175 non-null  object 
 1   Incident Number              865175 non-null  int64  
 2   Report Type                  865089 non-null  object 
 3   Report Type Description      805823 non-null  object 
 4   Incident Status Code         865155 non-null  object 
 5   Incident Status Description  865145 non-null  object 
 6   Investigation Status         865175 non-null  object 
 7   Incident Occurred            865175 non-null  object 
 8   Incident Reported            865175 non-null  object 
 9   Incident Location            859872 non-null  object 
 10  Latitude                     848238 non-null  float64
 11  Longitude                    848238 non-null  float64
 12  ZIP Code                     454148 non-null  float64
 13 

In [8]:
#pulled in the only columns I needed from the entire police inident sheet
officers = df[['Report Type Description','Investigation Status','Incident Occurred','Incident Reported','ZIP Code']]
officers


,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code
0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0
1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0
2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0
3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0
4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0
...,...,...,...,...,...
865170,DISPATCHED,Closed,2/15/2024 8:00,2/15/2024 10:12,37211.0
865171,DISPATCHED,Open,2/13/2024 22:30,2/14/2024 12:32,NaN
865172,DISPATCHED,Open,2/16/2024 10:00,2/16/2024 15:22,NaN
865173,DISPATCHED,Open,2/15/2024 5:00,2/16/2024 0:03,NaN


In [14]:
mnpd_officers = officers.loc[officers['Report Type Description'] == "DISPATCHED"]
mnpd_officers



,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code
0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0
1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0
2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0
3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0
4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0
...,...,...,...,...,...
865170,DISPATCHED,Closed,2/15/2024 8:00,2/15/2024 10:12,37211.0
865171,DISPATCHED,Open,2/13/2024 22:30,2/14/2024 12:32,NaN
865172,DISPATCHED,Open,2/16/2024 10:00,2/16/2024 15:22,NaN
865173,DISPATCHED,Open,2/15/2024 5:00,2/16/2024 0:03,NaN


In [16]:
npd_officers = mnpd_officers.loc[officers['Investigation Status'] == "Closed"]
npd_officers

,Report Type Description,Investigation Status,Incident Occurred,Incident Reported,ZIP Code
0,DISPATCHED,Closed,7/1/2019 17:31,7/1/2019 18:17,37214.0
1,DISPATCHED,Closed,3/31/2018 12:21,3/31/2018 19:41,37013.0
2,DISPATCHED,Closed,4/6/2020 11:15,4/6/2020 13:11,37207.0
3,DISPATCHED,Closed,5/29/2017 2:33,5/29/2017 4:27,37205.0
4,DISPATCHED,Closed,12/16/2017 18:30,12/16/2017 22:44,37076.0
...,...,...,...,...,...
865161,DISPATCHED,Closed,1/31/2024 19:00,1/31/2024 23:07,37115.0
865164,DISPATCHED,Closed,2/9/2024 10:17,2/9/2024 14:09,37013.0
865167,DISPATCHED,Closed,11/7/2023 15:58,11/7/2023 18:34,37203.0
865168,DISPATCHED,Closed,2/11/2024 14:00,2/11/2024 16:15,37204.0
